<a href="https://colab.research.google.com/github/sarahvx/navegador-terminal-python/blob/main/Navegador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Grupo:** Garotes de Programa


**Participantes:** Sarah Ferreira, Augusto Emanuel, Murilo Jansen

In [7]:
class Site:
  def __init__(self, link=None, prox=None):       # Inicializa um nó (site) com o link e a referência para o próximo site
    self.__link = link
    self.__prox = prox

  @property
  def link(self):
    return self.__link

  @property
  def prox(self):
    return self.__prox

  @prox.setter
  def prox(self, novo_prox):
    self.__prox = novo_prox

  def __str__(self):
    return str(self.__link)

In [8]:
class HistoricoPilha:

    def __init__(self):
        self.__topo = None

    @property
    def topo(self):
      return self.__topo

    def inserir_site(self, novo_site):    # Insere um novo site no topo da pilha do histórico
        novo = Site(novo_site)
        if self.__topo is None:
            self.__topo = novo
        else:
            novo.prox = self.__topo
            self.__topo = novo

    def remover_ultimo_site(self):    # Remove o site do topo da pilha (voltar no histórico)
        if self.__topo is None:
            return 'histórico vazio'
        else:
            site_a_remover = self.__topo
            self.__topo = self.__topo.prox
        return site_a_remover.link

    def mostrar_historico(self):     # Retorna todos os links armazenados na pilha (do topo até a base)
        historico = []
        atual = self.__topo
        while atual is not None:
            historico.append(atual.link)
            atual = atual.prox
        return historico

In [9]:
class PaginaNaoEncontradaException(Exception):    # Exceção para quando a URL digitada não existe no banco de sites

    def __init__(self, mensagem=''):
        super().__init__(mensagem)

class NaoHaPaginaAnteriorException(Exception):    # Exceção para quando o usuário tenta voltar mas não há histórico

    def __init__(self, mensagem=''):
      super().__init__(mensagem)


In [10]:
conteudos =  {'www.ifpb.edu.br': 'Bem vindo ao IFPB',
              'www.ifpb.edu.br/tsi': 'Lorem Ipsum',
              'www.ifpb.edu.br/tsi/professores': 'Bem vindo a área dos professores',
              'www.ifpb.edu.br/tsi/alunos': 'Bem vindo a área dos alunos',
              'www.ifpb.edu.br/rc': 'Lorem Ipsum',
              'www.ifpb.edu.br/rc/coordenacao': 'Bem vindo a coordenacão',
              'www.ifpb.edu.br/rc/matriz_curricular': 'Lorem Ipsum',
              'www.youtube.com': 'Bem vindo ao youtube ',
              'www.youtube.com/feed': 'Bem vindo ao seu feed',
              'www.youtube.com/subscriptions': 'Bem vindo a sua área de inscrições',            # Conteúdos de cada site
              'www.x.com': 'Bem vindo ao Twitter',
              'www.x.com/home': 'Bem vindo a sua home do Twitter',
              'www.x.com/explore': 'Bem vindo ao seu explorar',
              'www.reddit.com': 'Bem vindo ao Reddit',
              'www.reddit.com/popular': 'Bem vindo as páginas populares do Reddit',
              'www.reddit.com/gaming': 'Bem vindo a área gamer',
              'www.github.com': 'Bem vindo ao Github',
              'www.github.com/dashboard': 'Bem vindo ao dashboard',
              'www.github.com/projects': 'Bem vindo aos seus projetos',
              'www.instagram.com': 'Bem vindo ao Instagram',
              'www.instagram.com/explore': 'Bem vindo ao seu explorar',
              'www.instagram.com/reels': 'Bem vindo aos reels'}


In [11]:
class Browser:

  def __init__(self, urls_iniciais, conteudos):
      self.__historico = HistoricoPilha()
      self.__home = ''
      self.__urls = urls_iniciais
      self.__conteudos = conteudos
      self.__historico_completo = []

  @property
  def historico(self):         # Retorna os últimos 3 sites do histórico (pilha)
      historico_parcial = []
      atual = self.__historico.topo
      while atual is not None:
        historico_parcial.append(atual.link)
        atual = atual.prox

      return historico_parcial[:3]

  @property
  def home(self):  # Página atual
      return self.__home

  def visitar_site(self, url):  # Acessa um site a partir da URL
      url = url.strip()
      if url not in urls_linhas:  # verifica se a URL existe
          raise PaginaNaoEncontradaException('Erro 404: Página não encontrada')
      else:
        self.__historico_completo.append(url)    # salva no histórico completo
        print('Página encontrada!\n')

        print(f'{self.__conteudos.get(url, 'Nenhum conteúdo disponível.')}\n')   # mostra o conteúdo do site
        print("Links disponíveis\n")

        links_disponiveis = []
        for linha in urls_linhas:
            if linha.startswith(url + '/'):
                partes = linha[len(url):].strip('/').split('/')    # mostra os links internos disponíveis
                if partes:
                    link = '/' + partes[0]
                    if link not in links_disponiveis:
                        links_disponiveis.append(link)
        for link in links_disponiveis:
          print(link)

      if self.__home != '':
          self.__historico.inserir_site(self.__home)
          self.__home = url
      else:
          self.__home = url

  def showhist(self):   # Retorna o histórico completo
      if self.__historico_completo == []:
        return 'Histórico vazio.'
      else:
        return self.__historico_completo[::-1]

  def back(self):   # Volta para a página anterior
      if self.__home == '':
        raise NaoHaPaginaAnteriorException('Não há página anterior.')
      site_pro_home = self.__historico.remover_ultimo_site()   # remove o site atual do histórico (pilha)
      if site_pro_home != 'histórico vazio':
          self.__home = site_pro_home   # define o anterior como home
      else:
          self.__home = ''

      print(f'\n{self.__conteudos.get(self.__home, "Nenhum conteúdo disponível.")}\n')    # exibe o conteúdo e os links disponíveis

      links_disponiveis = []
      for linha in urls_linhas:
          if linha.startswith(self.__home + '/'):
              partes = linha[len(self.__home):].strip('/').split('/')
              if partes:
                  link = '/' + partes[0]
                  if link not in links_disponiveis:
                      links_disponiveis.append(link)
      if links_disponiveis:
          print("Links disponíveis:")
          for link in links_disponiveis:
              print(link)

  def adicionar_novo_site(self, nova_url: str):   # Adiciona uma nova URL ao banco de sites, caso não exista
    nova_url = nova_url.strip()
    if nova_url not in self.__urls:
      urls_linhas.append(nova_url)


In [12]:
#URLS INICIAIS

urls = open("urls.txt", "r")
urls_data = urls.read()
urls_linhas = [linha.strip() for linha in urls_data.splitlines()]


# INTERFACE
navegador = Browser(urls_linhas, conteudos)
while True:
    print('______________________________________________________________________')
    print('Use o comando #help para ver todos os comandos disponíveis.')
    print('Histórico:', navegador.historico)
    print('Home: ', navegador.home)
    print('Digite a url ou digite #back para retornar à última página visitada.')
    url = str(input('url: '))


    if url == '#help':
      print('\n#add <site> = Adiciona um novo site ao banco de dados.')
      print('#back = Retorna à última página visitada.')
      print('#showhist = Mostra o histórico completo.')
      print('#sair = Encerra o programa.')

    elif url.startswith('#add'):
      try:
        comando, site = url.split(' ', 1)
        if site == '':
          print('Formato incorreto. Use: #add <site>')
        elif site in urls_linhas:
          print('Site já cadastrado.')
        elif site.startswith('www'):
          navegador.adicionar_novo_site(site)
          print('Site adicionado com sucesso!')
        else:
          print('Formato incorreto. O site deve começar com "www".')
      except ValueError:
            print('Formato incorreto. Use: #add <site>')

    elif url == '#back':
        try:
          navegador.back()
        except NaoHaPaginaAnteriorException as e:
          print(e)

    elif url == '#showhist':
        print('\nHISTÓRICO:')
        print(navegador.showhist())

    elif url == '#sair':
        print('Fechando o programa...')
        break

    elif url.startswith('/'):
        if navegador.home != '':
          url_completa = navegador.home + url
          navegador.visitar_site(url_completa)
        else:
          print('Tente com uma URL completa.')

    else:
      try:
        navegador.visitar_site(url)
      except PaginaNaoEncontradaException as e:
        print(e)

______________________________________________________________________
Use o comando #help para ver todos os comandos disponíveis.
Histórico: []
Home:  
Digite a url ou digite #back para retornar à última página visitada.
url: #help

#add <site> = Adiciona um novo site ao banco de dados.
#back = Retorna à última página visitada.
#showhist = Mostra o histórico completo.
#sair = Encerra o programa.
______________________________________________________________________
Use o comando #help para ver todos os comandos disponíveis.
Histórico: []
Home:  
Digite a url ou digite #back para retornar à última página visitada.
url: www.ifpb.edu.br
Página encontrada!

Bem vindo ao IFPB

Links disponíveis

/tsi
/rc
______________________________________________________________________
Use o comando #help para ver todos os comandos disponíveis.
Histórico: []
Home:  www.ifpb.edu.br
Digite a url ou digite #back para retornar à última página visitada.
url: /tsi
Página encontrada!

Lorem Ipsum

Links disp